<a href="https://colab.research.google.com/github/Gursharan-Reddy/CODTECH/blob/main/RateMyStay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install gradio tensorflow scikit-learn joblib pillow numpy pandas

In [18]:
# =============================================================
# RateMyStay — AI Hotel Review Analyzer (2025 version)
# =============================================================
# Works directly in Google Colab
#   1. Run this cell
#   2. Click the Gradio link shown after "Running on public URL:"
# =============================================================

!pip install gradio scikit-learn tensorflow pillow numpy pandas textblob --quiet

import gradio as gr
import numpy as np
import pandas as pd
import re
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import tensorflow as tf

# =============================================================
# 1. TRAIN BASIC SENTIMENT & RATING MODELS ON SAMPLE DATA
# =============================================================
sample_reviews = [
    "The hotel was clean and staff were friendly. Loved my stay!",
    "Room was dirty, air conditioning broken, and rude staff.",
    "Average service, not bad but not great either.",
    "Excellent food and beautiful view from the balcony.",
    "Worst experience ever. Dirty sheets and cold food.",
    "Everything was perfect. Highly recommend this hotel!",
    "The noise at night was terrible and bathroom was filthy.",
    "The pool was amazing, and the breakfast was delicious.",
    "Mediocre stay, okay for one night.",
    "Terrible service, smelly room, never coming back!"
]
sample_labels = [5, 1, 3, 5, 1, 5, 2, 5, 3, 1]

vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
X_train = vectorizer.fit_transform(sample_reviews)
rating_model = LogisticRegression(max_iter=1000).fit(X_train, sample_labels)
sent_model = LogisticRegression(max_iter=1000).fit(X_train, np.sign(np.array(sample_labels) - 3))

# =============================================================
# 2. LOAD IMAGE MODEL (for optional photo classification)
# =============================================================
base_model = MobileNetV2(weights="imagenet", include_top=True)

# =============================================================
# 3. IMPROVED SENTIMENT ANALYSIS FUNCTION
# =============================================================
def predict_sentiment(review_text):
    """Combine ML prediction + keyword boost + polarity check."""
    if not review_text or str(review_text).strip() == "":
        return "Neutral", 0.5

    X = vectorizer.transform([review_text])
    probs = sent_model.predict_proba(X)[0]
    pos_prob = float(probs[1])
    text_lower = review_text.lower()

    neg_keywords = [
        "disaster","never again","terrible","awful","dirty","filthy",
        "smelled","stained","noisy","horrible","overpriced","worst",
        "broken","bugs","cockroach","unacceptable","smoke","cold food"
    ]
    pos_keywords = [
        "wonderful","excellent","amazing","great","clean","friendly",
        "comfortable","delicious","fantastic","perfect","awesome"
    ]
    neg_hits = sum(1 for k in neg_keywords if k in text_lower)
    pos_hits = sum(1 for k in pos_keywords if k in text_lower)

    # Apply penalty/boost
    if neg_hits > 0:
        pos_prob *= 0.4 ** min(neg_hits, 2)
    elif pos_hits > 0:
        pos_prob = 1 - ((1 - pos_prob) * 0.5 ** min(pos_hits, 2))

    # Blend with TextBlob polarity for subtlety
    blob_polarity = TextBlob(review_text).sentiment.polarity
    pos_prob = (pos_prob * 0.7) + ((blob_polarity + 1) / 2 * 0.3)

    if pos_prob >= 0.6:
        label = "Positive"
    elif pos_prob <= 0.4:
        label = "Negative"
    else:
        label = "Neutral"
    return label, pos_prob

# =============================================================
# 4. RATING PREDICTION LOGIC
# =============================================================
def predict_rating(review_text, nights, room_type, img_flag):
    """Predict star rating from sentiment + details."""
    sent_label, pos_prob = predict_sentiment(review_text)
    base_rating = 3 + (pos_prob - 0.5) * 4  # shift from 1–5 roughly
    # Room quality adds small positive weight
    room_bonus = {"Standard": 0, "Deluxe": 0.3, "Suite": 0.5}[room_type]
    rating = base_rating + room_bonus
    if img_flag == 1 and sent_label == "Negative":
        rating -= 0.5  # penalize visible evidence
    rating = int(np.clip(round(rating), 1, 5))
    return rating, sent_label, pos_prob

# =============================================================
# 5. IMAGE ANALYSIS
# =============================================================
def analyze_image(img):
    if img is None:
        return "No image uploaded", 0
    img = img.convert("RGB").resize((224,224))
    x = np.expand_dims(image.img_to_array(img), axis=0)
    x = preprocess_input(x)
    preds = base_model.predict(x)
    label = decode_predictions(preds, top=1)[0][0][1]
    evidence_flag = 1 if any(word in label.lower() for word in ["dirty","mess","couch","stain"]) else 0
    return label, evidence_flag

# =============================================================
# 6. MANAGERIAL ACTIONS
# =============================================================
def recommend_action(sentiment, rating, evidence_flag):
    if sentiment == "Negative" and rating <= 2:
        if evidence_flag:
            return "Urgent: Send housekeeping, inspect room, and offer refund/discount."
        else:
            return "Apologize and offer a small goodwill gesture."
    elif sentiment == "Positive" and rating >= 4:
        return "Send thank-you message and request public review."
    elif sentiment == "Neutral" or rating == 3:
        return "Request more feedback and offer small discount on next stay."
    else:
        return "Manual review recommended."

# =============================================================
# 7. MAIN ANALYSIS PIPELINE
# =============================================================
def analyze_review(review_text, stay_nights, room_type, image_input):
    img_label, evidence_flag = analyze_image(image_input)
    rating, sentiment, pos_prob = predict_rating(review_text, stay_nights, room_type, evidence_flag)
    rec = recommend_action(sentiment, rating, evidence_flag)
    return {
        "Predicted Star Rating": rating,
        "Sentiment": sentiment,
        "Sentiment (positive_prob)": round(pos_prob, 3),
        "Detected Image Label": img_label,
        "Image Evidence Flag": evidence_flag,
        "Recommended Manager Action": rec
    }

# =============================================================
# 8. GRADIO INTERFACE
# =============================================================
with gr.Blocks(title="RateMyStay — AI Hotel Review Analyzer") as demo:
    gr.Markdown("## 🏨 RateMyStay — AI Hotel Review Analyzer")
    gr.Markdown(
        "Paste a review, optionally upload a photo, and provide stay details.\n"
        "The app predicts the likely star rating, the sentiment, shows detected "
        "image label hints, and suggests a managerial action."
    )

    with gr.Row():
        review_input = gr.Textbox(label="Customer Review Text", placeholder="Enter the customer's review...")
        image_input = gr.Image(label="Optional Review Photo (photo of room/food)", type="pil")

    with gr.Row():
        stay_nights = gr.Slider(1, 10, value=2, step=1, label="Number of Nights Stayed")
        room_type = gr.Dropdown(["Standard", "Deluxe", "Suite"], label="Room Type", value="Standard")

    analyze_button = gr.Button("Analyze Review", elem_id="analyze-btn")
    output = gr.JSON(label="AI Analysis Results")

    analyze_button.click(analyze_review, inputs=[review_input, stay_nights, room_type, image_input], outputs=[output])

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5a640fc37f3efbbbf0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
